In [2]:
import cv2
import numpy as np
import sys

### オブジェクト除去
左の鸚鵡を消す

In [3]:
img = cv2.imread("img/Parrots.bmp")
h, w ,c = img.shape
mask = np.zeros((h, w, c), np.uint8)
cv2.imshow("Original", img)

cv2.rectangle(mask, (90, 0), (0, 480), (255, 255, 255), -1)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
cv2.imshow("Mask", mask)
dst = cv2.inpaint(img, mask, 1, cv2.INPAINT_TELEA)
cv2.imshow("Inpaint", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 画像修復
画像に入った白線を消す

In [11]:
img = cv2.imread("img/Pepper-inpaint.jpg")
mask = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(mask, 220, 255, cv2.THRESH_BINARY)
dst = cv2.inpaint(img, mask, 1, cv2.INPAINT_TELEA)
cv2.imshow("Original", img)
cv2.imshow("Mask", mask)
cv2.imshow("Inpaint", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### コーナー検出

In [14]:
img = cv2.imread("img/milkdrop.bmp")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
dst = img.copy()
corners = cv2.goodFeaturesToTrack(img_gray, 70, 0.08, 10)
for i in corners:
    x = int(i[0][0])
    y = int(i[0][1])
    cv2.circle(dst, (x, y), 3, (0, 255, 0), 2)
cv2.imshow("Original", img)
cv2.imshow("Corners", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### AKAZE
元画像・傾けた画像の特徴点を検出し、マッチング

In [16]:
img1 = cv2.imread("img/Parrots.bmp", 0)
h1, w1 = img1.shape
center = (int(w1/2), int(h1/2))
trans = cv2.getRotationMatrix2D(center, 30.0, 1.0)
img2 = cv2.warpAffine(img1, trans, (w1, h1))
h2, w2 = img2.shape
detertor = cv2.AKAZE_create()
kpts1, desc1 = detertor.detectAndCompute(img1, None)
kpts2, desc2 = detertor.detectAndCompute(img2, None)
matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = matcher.match(desc1, desc1)
dst1 = np.zeros((h1, w1+w1, 3), np.uint8)
cv2.drawMatches(img1, kpts1, img1, kpts1, matches, dst1)
cv2.imshow("Matches", dst1)

matches = matcher.match(desc1, desc2)
dst2 = np.zeros((max(h1, h2), w1+w2, 3), np.uint8)
cv2.drawMatches(img1, kpts1, img2, kpts2, matches, dst2)
cv2.imshow("Matches", dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### ORB

In [17]:
img1 = cv2.imread("img/Parrots.bmp", 0)
h1, w1 = img1.shape
center = (int(w1/2), int(h1/2))
trans = cv2.getRotationMatrix2D(center, 30.0, 1.0)
img2 = cv2.warpAffine(img1, trans, (w1, h1))
h2, w2 = img2.shape
detertor = cv2.ORB_create()
kpts1, desc1 = detertor.detectAndCompute(img1, None)
kpts2, desc2 = detertor.detectAndCompute(img2, None)
matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = matcher.match(desc1, desc1)
dst1 = np.zeros((h1, w1+w1, 3), np.uint8)
cv2.drawMatches(img1, kpts1, img1, kpts1, matches, dst1)
cv2.imshow("Matches", dst1)

matches = matcher.match(desc1, desc2)
dst2 = np.zeros((max(h1, h2), w1+w2, 3), np.uint8)
cv2.drawMatches(img1, kpts1, img2, kpts2, matches, dst2)
cv2.imshow("Matches", dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### ORB - 上位50点をマッチング

In [19]:
img1 = cv2.imread("img/Parrots.bmp", 0)
h1, w1 = img1.shape
center = (int(w1/2), int(h1/2))
trans = cv2.getRotationMatrix2D(center, 30.0, 1.0)
img2 = cv2.warpAffine(img1, trans, (w1, h1))
h2, w2 = img2.shape
detertor = cv2.ORB_create()
kpts1, desc1 = detertor.detectAndCompute(img1, None)
kpts2, desc2 = detertor.detectAndCompute(img2, None)
matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = matcher.match(desc1, desc2)
matches = sorted(matches, key = lambda x:x.distance)
dst = np.zeros((max(h1, h2), w1+w2, 3), np.uint8)
cv2.drawMatches(img1, kpts1, img2, kpts2, matches[:10], dst)
cv2.imshow("Matches", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

## テンプレートマッチング

### 準備

In [20]:
tmp = cv2.imread("img/Parrots.bmp")
cv2.imwrite("img/template.jpg", tmp[100:200, 100:200])


True

### テンプレートマッチング

In [22]:
img = cv2.imread("img/Parrots.bmp", 0)
tmp = cv2.imread("img/template.jpg", 0)
h, w = tmp.shape
res = cv2.matchTemplate(img, tmp, cv2.TM_SQDIFF)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
top_left = min_loc 
bottom_right = (top_left[0]+w, top_left[1]+h)
cv2.rectangle(img, top_left, bottom_right, 255, 2)
cv2.imshow("Match", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### カスケード分類器

In [ ]:
%reset -f
import cv2

### 顔検出

In [23]:
img = cv2.imread("img/Lenna.bmp")
cascade = cv2.CascadeClassifier("cascade/haarcascade_frontalface_alt.xml")
face = cascade.detectMultiScale(img)

if len(face) > 0:
    for x, y, w, h in face:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
else:
    print("No face detected")

cv2.imshow("Face", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 目の検出

In [31]:
img = cv2.imread("img/Lenna.bmp")
cascade = cv2.CascadeClassifier("cascade/haarcascade_eye.xml")
face = cascade.detectMultiScale(img)

if len(face) > 0:
    for x, y, w, h in face:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
else:
    print("No face detected")

cv2.imshow("Face", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## YOLO

In [9]:
%reset -f
import cv2
import numpy as np
from ultralytics import YOLO

### 画像分類

In [14]:
model = YOLO("cascade/yolov8n-cls.pt")
# C:\Users\daban\Documents\schoolfiles\img\PXL_20241110_020044637.jpg
img = cv2.imread("img/Lenna.bmp", cv2.IMREAD_COLOR)
results = model.predict(img, conf=0.5)
result = results[0]
plot = result.plot()
cv2.imshow("YOLO", plot)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 224x224 cowboy_hat 0.44, sombrero 0.39, brassiere 0.09, mask 0.01, bonnet 0.01, 16.9ms
Speed: 2.1ms preprocess, 16.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


### セグメンテーション

In [16]:
model = YOLO("cascade/yolov8n-seg.pt")
img = cv2.imread("img/bus.jpg")
results = model.predict(img, save=True, show_boxes = False)
result = results[0]
plot = result.plot()
cv2.namedWindow("result", cv2.WINDOW_NORMAL)
cv2.imshow("result", plot)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 640x480 4 persons, 1 bus, 1 skateboard, 95.3ms
Speed: 2.9ms preprocess, 95.3ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to c:\Users\daban\Documents\schoolfiles\runs\segment\predict16


### 骨格推定

In [15]:
model = YOLO("cascade/yolov8n-pose.pt")
img = cv2.imread("img/bus.jpg")
results = model.predict(img)
result = results[0]
plot = result.plot()
cv2.namedWindow("result", cv2.WINDOW_NORMAL)
cv2.imshow("result", plot)
cv2.waitKey(0)
cv2.destroyAllWindows()

100%|██████████| 6.52M/6.52M [00:00<00:00, 14.4MB/s]



0: 640x480 4 persons, 74.8ms
Speed: 3.1ms preprocess, 74.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
